In [1]:
import pandas as pd
from datetime import date,datetime
from dateutil.relativedelta import relativedelta
import sys
import os
import json

In [22]:
df=pd.read_csv('../quote_v1.csv').assign(len_quote=lambda x:x.quote.str.len()).sample(frac=1)[['quote','author','big_cate','tag']]

In [24]:
df=df.assign(num_word=lambda x:x.quote.str.split(' '))\
    .assign(num_word=lambda x:x.num_word.str.len())\
    .query('num_word>=5 and num_word<=9').reset_index()

In [25]:
dt_start=date(2023,5,1)
for i,dt in enumerate(pd.date_range(dt_start,dt_start+relativedelta(days=5000))):
    row=df.iloc[i%len(df)]
    year_month=dt.strftime('%Y%m')
    year_month_directory='../data/features/dailyQuote/{}'.format(year_month)
    current_dt_str=dt.strftime('%Y%m%d')
    if not os.path.exists(year_month_directory):
        os.makedirs(year_month_directory)
    with open(
                "../data/features/dailyQuote/{}/{}.json".format(year_month,current_dt_str), "w"
            ) as outfile:
        json.dump(
            {
                'content':{
                    'item':row.quote,
                    'owner':row.author
                },
                'updatedAt':datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            },
            outfile
        )
    